In [1]:
import numpy as np
import pandas as pd
import seaborn as sns  # for heatmaps
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from collections import Counter
%matplotlib inline

In [2]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from torchsummary import summary
import torchvision

# Contrastive_learning_for_unseen_environments

In [3]:
# random seed
np.random.seed(1024)
torch.manual_seed(1024)

# gpu setting
DEVICE = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
device = DEVICE
torch.cuda.set_device(DEVICE)

## 0. Setup 

In [4]:
from data.spectrogram import import_data, import_pair_data
from data import *

In [5]:
from models.baseline import *
from models.cnn import *
from models.self_supervised import *
from models.utils import *
from models import *

In [6]:
from losses import NT_Xent
from contrastive_learning import pretrain
from train import record_log,evaluation,train,load_checkpoint,save_model

In [7]:
def load_encoder(network, model_fp):
    encoder, outsize = create_encoder(network,'nuc2')
    encoder.load_state_dict(torch.load(model_fp))
    return encoder, outsize

def load_model(network, freeze, lb, model_fp):
    encoder, outsize = create_encoder(network,'nuc2')
    model = add_classifier(encoder,in_size=outsize,out_size=len(lb.classes_),freeze=freeze)
    model.load_state_dict(torch.load(model_fp))
    return model
    

# Main

In [26]:
TRAINMODE = 'normal' #  'simclr' #        
NETWORK =  'shallow2v2' #  'alexnet' # 'vgg16' # 'resnet' # 'shallow' #      
PAIRING =  'nuc2' # 'pwr' #  
TEMPERATURE = 'L' 

REGULARIZE = None
JOINT = 'first'

PRETRAIN_SKIP = ['waving'] # 'standff'
FINETUNE_SKIP = ['waving'] # 'waving','waving'

EXTRA = 'p6bf6b'

In [27]:
fp2 = './data/experiment_data/exp_2/spectrogram' 
fp3 = './data/experiment_data/exp_3/spectrogram' 
fp4 = './data/experiment_data/exp_4/spectrogram_multi'
OUT_PATH = './laboratory/experiment_b'

BATCH_SIZE = 64
NUM_WORKERS = 2
PRETRAIN_EPOCHS = 750
LAB_FINETUNE_EPOCHS = 200
FIELD_FINETUNE_EPOCHS = 200
Y_SAMPLING = None

In [28]:
trainmode = TRAINMODE
pairing = PAIRING
network = NETWORK
temperature = TEMPERATURE
joint = JOINT
pretrain_skip = PRETRAIN_SKIP
activities = FINETUNE_SKIP
batch_size = BATCH_SIZE
num_workers = NUM_WORKERS
pretrain_epochs = PRETRAIN_EPOCHS 
lab_finetune_epochs = LAB_FINETUNE_EPOCHS 
field_finetune_epochs = FIELD_FINETUNE_EPOCHS
regularize = REGULARIZE 
t = TEMPERATURE
y_sampling = Y_SAMPLING

temperature_dict = {'L':0.1,'M':0.5,'H':1}    
temperature = temperature_dict[t]

if TRAINMODE == 'pwr':
    joint = 'first'

if TRAINMODE == 'normal':
    if regularize == True:
        t = 'r'
    else:
        t = ''
    pairing = 'nuc2'
    freeze=False
else:
    freeze=True

if JOINT == 'joint':
    j='-j'
else:
    j=''
    
if EXTRA == None:
    extra = ''
else:
    extra = '-Extra-'+str(EXTRA)


exp_name = f'Trainmode-{trainmode}-{t}_Network-{network}_Data-exp4csi{pairing}{j}{extra}'
model_outpath  = OUT_PATH+'/'+'saved_models'
record_outpath = OUT_PATH+'/'+'records'

print(exp_name)

Trainmode-normal-_Network-shallow2v2_Data-exp4csinuc2-Extra-p6bf6b


In [11]:
####################################################### Main #############################################################

datas = import_pair_data(fp4,modal=['csi',pairing])

Importing Data >>>>>>> Complete


In [12]:
X1,X2,y = initial_filtering_activities(datas,activities=pretrain_skip)

In [13]:
X1_train, X1_test, X2_train, X2_test, y_train_, y_test_ = split_datasets(X1,X2,y,split=0.8,stratify=y)

In [19]:
################################ Lab-Pretrain-phase ################################

### data
pretrain_loader = create_dataloader(X1_train,X2_train,batch_size=batch_size,num_workers=num_workers)
print('X1_train: ',X1_train.shape,'\tX2_train: ',X2_train.shape)

### network
encoder, outsize = create_encoder(network,'nuc2')
encoder2, outsize2 = None,None

if joint == 'joint':
    simclr = add_SimCLR(encoder, outsize)
else:
    encoder2, outsize2 = create_encoder(network,pairing)
    simclr = add_SimCLR_multi(enc1=encoder,enc2=encoder2,out_size1=outsize,out_size2=outsize2)
    
    
    
    

### pretraining
phase = 'pretrain'
if trainmode == 'simclr':
    criterion = NT_Xent(batch_size, temperature, world_size=1)
    optimizer = torch.optim.SGD(list(simclr.parameters()), lr=0.0005)
    simclr, record = pretrain(model=simclr,
                              train_loader=pretrain_loader,
                              criterion=criterion,
                              optimizer=optimizer,
                              end=pretrain_epochs,
                              device=device)
    record_log(record_outpath,exp_name,phase,record=record)
    
# save
encoder_fp = save_model(model_outpath,exp_name,phase,simclr.encoder)
del simclr,encoder,outsize, encoder2,outsize2
torch.cuda.empty_cache()

X1_train:  (477, 1, 65, 501) 	X2_train:  (477, 1, 65, 501)


In [20]:
################################ Lab-Finetuning-phase ################################
samplings = [1,5,10,'weight','undersampling','oversampling',]
#samplings = ['undersampling','oversampling']

inital = {'lab':True,'field':True}
for sampling in samplings:
    
    print('\n\nsampling: ',sampling,'\n')
    
    ### Sampling data ###
    X_train, X_test, y_train, y_test = select_train_test_dataset(X1_train, X1_test, X2_train, X2_test, y_train_, y_test_, joint)
    X_train, X_test, y_train, y_test, lb = filtering_activities_and_label_encoding(X_train, X_test, y_train, y_test, activities)
    lab_finetune_loader, lab_validatn_loader, class_weight = combine1(X_train, X_test, y_train, y_test, 
                                                                      sampling, lb, batch_size, num_workers, 
                                                                      y_sampling=y_sampling)
    print("class: ",lb.classes_)
    print("class_size: ",1-class_weight)
    
    ### model 
    encoder, outsize = load_encoder(network, encoder_fp)
    model = add_classifier(encoder,in_size=outsize,out_size=len(lb.classes_),freeze=freeze)
    
    # initialization
    phase = 'lab-initial'
    if inital['lab']:
        cmtx,cls = evaluation(model,lab_finetune_loader,label_encoder=lb)
        record_log(record_outpath,exp_name,phase,cmtx=cmtx,cls=cls)
        inital['lab'] = False
    
    # finetuning 
    phase = 'lab-finetune'+'-'+str(sampling)
    criterion = nn.CrossEntropyLoss(weight=class_weight).to(device)
    optimizer = torch.optim.Adam(list(model.parameters()), lr=0.0005)
    model, record = train(model=model,
                          train_loader= lab_finetune_loader,
                          criterion=criterion,
                          optimizer=optimizer,
                          end= lab_finetune_epochs,
                          test_loader = lab_validatn_loader,
                          device = device,
                          regularize = regularize)
    
    # record and save
    cmtx,cls = evaluation(model,lab_validatn_loader,label_encoder=lb)
    record_log(record_outpath,exp_name,phase,record=record,cmtx=cmtx,cls=cls,acc_rec=True)
    
    if sampling != 'weight':

        del encoder,model,criterion,optimizer# ,record,cmtx,cls
        torch.cuda.empty_cache()

    elif sampling == 'weight':
        
        model_fp = save_model(model_outpath,exp_name,phase,model)
        del encoder,model,criterion,optimizer# ,record,cmtx,cls
        torch.cuda.empty_cache()
    



sampling:  1 

X_train:  (6, 1, 65, 501) 	y_train:  (6,) 	X_test:  (120, 1, 65, 501) 	y_test:  (120,)
class:  ['lay' 'pickup' 'sit' 'stand' 'standff' 'walk']
class_size:  tensor([0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667])
             0    1    2     3         4    5  accuracy  macro avg  \
f1-score   0.0  0.0  0.0  0.40  0.666667  0.0  0.333333   0.177778   
precision  0.0  0.0  0.0  0.25  0.500000  0.0  0.333333   0.125000   
recall     0.0  0.0  0.0  1.00  1.000000  0.0  0.333333   0.333333   
support    1.0  1.0  1.0  1.00  1.000000  1.0  0.333333   6.000000   

           weighted avg  
f1-score       0.177778  
precision      0.125000  
recall         0.333333  
support        6.000000  
Start Training
Epoch 1: 

/home/andy/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


> loss: 1.7916284799575806 
Epoch 2: > loss: 1.7782055139541626 
Epoch 3: > loss: 1.9418363571166992 
Epoch 4: > loss: 1.8186302185058594 
Epoch 5: > loss: 1.7999969720840454 
Epoch 6: > loss: 1.8294713497161865 
Epoch 7: > loss: 1.78205144405365 
Epoch 8: > loss: 1.7781826257705688 
Epoch 9: > loss: 1.7797338962554932 
Epoch 10: > loss: 1.767246961593628  accuracy: 0.25
Epoch 11: 

/home/andy/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


> loss: 1.7204627990722656 
Epoch 12: > loss: 1.6936477422714233 
Epoch 13: > loss: 1.5615367889404297 
Epoch 14: > loss: 1.3416565656661987 
Epoch 15: > loss: 1.2088096141815186 
Epoch 16: > loss: 1.474308729171753 
Epoch 17: > loss: 0.8448276519775391 
Epoch 18: > loss: 0.8890425562858582 
Epoch 19: > loss: 0.8391897082328796 
Epoch 20: > loss: 0.36191755533218384  accuracy: 0.4666666666666667
Epoch 21: > loss: 0.4361000657081604 
Epoch 22: > loss: 0.40980440378189087 
Epoch 23: > loss: 0.25358840823173523 
Epoch 24: > loss: 0.2802278697490692 
Epoch 25: > loss: 0.10860762745141983 
Epoch 26: > loss: 0.04200801998376846 
Epoch 27: > loss: 0.1735728532075882 
Epoch 28: > loss: 0.008719624020159245 
Epoch 29: > loss: 0.010665475390851498 
Epoch 30: > loss: 0.008904831483960152  accuracy: 0.45
Epoch 31: > loss: 0.011554531753063202 
Epoch 32: > loss: 0.016103357076644897 
Epoch 33: > loss: 0.004962273873388767 
Epoch 34: > loss: 0.003800180973485112 
Epoch 35: > loss: 0.0044701034203171

...

Epoch 128: >>>>>>> loss: 0.001568351755850017 
Epoch 129: >>>>>>> loss: 0.002378604607656598 
Epoch 130: >>>>>>> loss: 0.0006587313255295157  accuracy: 0.7083333333333334
Epoch 131: >>>>>>> loss: 0.0004512361774686724 
Epoch 132: >>>>>>> loss: 0.0006462421151809394 
Epoch 133: >>>>>>> loss: 8.327377872774377e-05 
Epoch 134: >>>>>>> loss: 0.00037356436951085925 
Epoch 135: >>>>>>> loss: 0.002102221129462123 
Epoch 136: >>>>>>> loss: 0.00013752507220488042 
Epoch 137: >>>>>>> loss: 0.00061996333533898 
Epoch 138: >>>>>>> loss: 0.00023308659729082137 
Epoch 139: >>>>>>> loss: 8.741179772187024e-05 
Epoch 140: >>>>>>> loss: 0.0005984487943351269  accuracy: 0.7166666666666667
Epoch 141: >>>>>>> loss: 1.682177207840141e-05 
Epoch 142: >>>>>>> loss: 3.487684443825856e-05 
Epoch 143: >>>>>>> loss: 0.00016911157581489533 
Epoch 144: >>>>>>> loss: 4.306021946831606e-05 
Epoch 145: >>>>>>> loss: 4.484940291149542e-05 
Epoch 146: >>>>>>> loss: 0.002215304411947727 
Epoch 147: >>>>>>> loss: 0.00012

/home/andy/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


>> loss: 1.5130113363265991 
Epoch 12: >> loss: 1.5833712816238403 
Epoch 13: >> loss: 1.5109893083572388 
Epoch 14: >> loss: 1.4857940673828125 
Epoch 15: >> loss: 1.5650359392166138 
Epoch 16: >> loss: 1.3761577606201172 
Epoch 17: >> loss: 1.3612818717956543 
Epoch 18: >> loss: 1.3748964071273804 
Epoch 19: >> loss: 1.4082597494125366 
Epoch 20: >> loss: 1.4037549495697021  accuracy: 0.5166666666666667
Epoch 21: >> loss: 1.3093256950378418 
Epoch 22: >> loss: 1.2563413381576538 
Epoch 23: >> loss: 1.223339557647705 
Epoch 24: >> loss: 1.0535532236099243 
Epoch 25: >> loss: 0.9931107759475708 
Epoch 26: >> loss: 1.0639467239379883 
Epoch 27: >> loss: 1.3275794982910156 
Epoch 28: >> loss: 1.169001579284668 
Epoch 29: >> loss: 1.0179144144058228 
Epoch 30: >> loss: 1.0639212131500244  accuracy: 0.6166666666666667
Epoch 31: >> loss: 1.0175299644470215 
Epoch 32: >> loss: 0.9747475385665894 
Epoch 33: >> loss: 0.9513515830039978 
Epoch 34: >> loss: 0.9870182275772095 
Epoch 35: >> loss:

                  0          1          2          3         4          5  \
f1-score   0.400000   0.487805   0.514286   0.400000  0.500000   0.872340   
precision  0.312500   0.526316   0.473684   0.500000  0.416667   0.931818   
recall     0.555556   0.454545   0.562500   0.333333  0.625000   0.820000   
support    9.000000  22.000000  16.000000  15.000000  8.000000  50.000000   

           accuracy   macro avg  weighted avg  
f1-score      0.625    0.529072      0.634811  
precision     0.625    0.526831      0.661622  
recall        0.625    0.558489      0.625000  
support       0.625  120.000000    120.000000  


sampling:  oversampling 

X_train:  (1188, 1, 65, 501) 	y_train:  (1188,) 	X_test:  (120, 1, 65, 501) 	y_test:  (120,)
class:  ['lay' 'pickup' 'sit' 'stand' 'standff' 'walk']
class_size:  tensor([0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667])
Start Training
Epoch 1: >>>>>>>>>>>>>>>>>> loss: 1.572352647781372 
Epoch 2: >>>>>>>>>>>>>>>>>> loss: 1.4893513917922974 
Epoch

Epoch 124: >>>>>>>>>>>>>>>>>> loss: 0.05176563188433647 
Epoch 125: >>>>>>>>>>>>>>>>>> loss: 0.05626789852976799 
Epoch 126: >>>>>>>>>>>>>>>>>> loss: 0.08869876712560654 
Epoch 127: >>>>>>>>>>>>>>>>>> loss: 0.03765399008989334 
Epoch 128: >>>>>>>>>>>>>>>>>> loss: 0.012035449966788292 
Epoch 129: >>>>>>>>>>>>>>>>>> loss: 0.0005959714762866497 
Epoch 130: >>>>>>>>>>>>>>>>>> loss: 0.0003537412849254906  accuracy: 0.675
Epoch 131: >>>>>>>>>>>>>>>>>> loss: 0.0031355239916592836 
Epoch 132: >>>>>>>>>>>>>>>>>> loss: 0.0015164125943556428 
Epoch 133: >>>>>>>>>>>>>>>>>> loss: 3.9093371015042067e-05 
Epoch 134: >>>>>>>>>>>>>>>>>> loss: 5.382402741815895e-05 
Epoch 135: >>>>>>>>>>>>>>>>>> loss: 0.00028713420033454895 
Epoch 136: >>>>>>>>>>>>>>>>>> loss: 0.00026747799711301923 
Epoch 137: >>>>>>>>>>>>>>>>>> loss: 0.0011456637876108289 
Epoch 138: >>>>>>>>>>>>>>>>>> loss: 0.0020725037902593613 
Epoch 139: >>>>>>>>>>>>>>>>>> loss: 1.6000154573703185e-05 
Epoch 140: >>>>>>>>>>>>>>>>>> loss: 0.0001065

In [ ]:
assert False

# Records

In [ ]:
# simclr

       'lay'      'pickup'   'sit'      'stand'     'standff'  'walk'    'waving'  

'full'  11         33         12         19                    99
'full'  12         34         11         19                    98         57
'full'  9          32         13         19          5         99
'full'  7          27         13         17          6         95         57    
    
#vgg16             #joint
'1pcs'  0.48       0.485981   0.153846   0.478873               0.805825
'5pcs'  0.062500   0.536082   0.410256   0.307692               0.931217
'10pcs' 0.612245   0.349206   0.354430   0.562500               0.953368
'full'  0.687500   0.666667   0.428571   0.622951               0.99

'1pcs'  0.512821   0.366197   0.400000   0.482759               0.563536   0.493151
'5pcs'  0.5        0.358974   0.400000   0.422535               0.666667   0.496000
'10pcs' 0.727273   0.623377   0.405797   0.448276               0.649485   0.521739 
'full'  0.727273   0.673267   0.407407   0.633333               0.942308   0.919355

'1pcs'  0.219178   0.041667   0.186047   0.506329    0.12500    0.613757
'5pcs'  0.461538   0.578947   0.318841   0.375000    0.342857   0.954315 
'10pcs' 0.387097   0.488372   0.493506   0.520000    0.461538   0.964467 
'full'  0.514286   0.666667   0.448276   0.622951    0.370370   0.975369

'1pcs'  0.214286   0.378947   0.2500     0.347826    0.272727   0.283688   0.457447
'5pcs'  0.228571   0.595745   0.2500     0.509091    0.448980   0.520833   0.359712
'10pcs' 0.5        0.539326   0.412698   0.400000    0.424242   0.682927   0.512397
'full'  0.437500   0.586957   0.426230   0.557377    0.400000   0.917874   0.883721

In [ ]:
# simclr

        'lay'      'pickup'   'sit'      'stand'     'standff'  'walk'    'waving' 
    
#vgg16             #first'    #L
'1pcs'  0.0        0.638297   0.3783783  0.222222    0.142857   0.328767   0.479166 
'5pcs'  0.526315   0.627450   0.5        0.476190    0.133333   0.66       0.441176
'10pcs' 0.777777   0.651162   0.5714285  0.5         0.526315   0.64       0.578947
'full'  0.666667   0.708333   0.6875     0.592593    0.363636   0.893204   0.835821
'under' 0.700000   0.714286   0.533333   0.482759    0.421053   0.755102   0.617647
'over'  0.736842   0.681818   0.647059   0.642857    0.461538   0.88       0.823529
                                
                              #M
'full'  0.571429   0.653061   0.666667   0.538462    0.631579   0.921569  0.878788
'under' 0.555556   0.600000   0.578947   0.500000    0.583333   0.835165  0.760563
'over'  0.625000   0.723404   0.600000   0.533333    0.823529   0.875000  0.828571

                              #H
'full'  0.428571   0.680851   0.647059   0.592593    0.588235   0.847826  0.826667 
'under' 0.526316   0.590909   0.571429   0.592593    0.608696   0.727273 0.675325
'over'  0.705882   0.638298   0.600000   0.571429    0.631579   0.891304  0.876712

                   #joint
'1pcs'  0.214286   0.378947   0.2500     0.347826    0.272727   0.283688   0.457447
'5pcs'  0.228571   0.595745   0.2500     0.509091    0.448980   0.520833   0.359712
'10pcs' 0.5        0.539326   0.412698   0.400000    0.424242   0.682927   0.512397
'full'  0.437500   0.586957   0.426230   0.557377    0.400000   0.917874   0.883721

                   #pwr
'full'  0.0        0.775510   0.588235   0.666667    0.0        0.944000   0.833333
'under' 0.4        0.618182   0.705882   0.583333    0.25       0.520833   0.368421
'over'  0.285714   0.760000   0.500000   0.666667    0.40       0.918033   0.806452


        

#shallow           #joint
'1pcs'  0.384615   0.037736   0.356164   0.046512    0.0        0.555556   0.268657
'5pcs'  0.146341   0.169492   0.156863   0.484211    0.266667   0.357616   0.470588
'10pcs' 0.408163   0.391753   0.222222   0.338983    0.186047   0.541935   0.468750
'full'  0.333333   0.444444   0.356164   0.406780    0.370370   0.94       0.892086

                   #first
'full'  0.500000   0.5        0.4375     0.580645    0.615385   0.923077   0.909091  
'under' 0.384615   0.514286   0.685714   0.787879    0.352941   0.800000   0.769231
'over'  0.400000   0.465116   0.470588   0.580645    0.571429   0.812500   0.794521


#alexnet
'1pcs'  0.0        0.068182   0.170213   0.421053    0.0        0.375940   0.551351
'5pcs'  0.274510   0.515464   0.393939   0.285714    0.166667   0.590909   0.525547
'10pcs' 0.615385   0.530120   0.41791    0.285714    0.476190   0.604651   0.575163 
'full'  0.592593   0.606061   0.433333   0.483871    0.5        0.920792   0.876923
 
                   #first
'full'  0.875000   0.681818   0.611111   0.571429    0.75       0.907216  0.898551
'under' 0.666667   0.615385   0.666667   0.689655    0.695652   0.775510  0.603175
'over'  0.823529   0.604651   0.457143   0.571429    0.777778   0.888889  0.848485

# Resnet           #first

'full'  0.400000   0.666667   0.611111   0.482759    0.428571   0.843137  0.823529
'under' 0.400000   0.450000   0.611111   0.444444    0.315789   0.747253  0.576271
'over'  0.526316   0.487805   0.550000   0.466667    0.307692   0.860215  0.828571

In [ ]:
# normal
        'lay'      'pickup'   'sit'      'stand'     'standff'  'walk'    'waving'  
#vgg16              #joint
'1pcs'  0.074074   0.192771   0.385542   0.256410    0.0        0.133333   0.055556
'5pcs'  0.378378   0.325581   0.204082   0.425000    0.210526   0.548571   0.359712
'10pcs' 0.0        0.385965   0.372093   0.235294    0.204082   0.205128   0.500000
'full'  0.421053   0.494382   0.327273   0.538462    0.648649   0.919431   0.876923

                    #first
'full'  0.714286   0.415094   0.258065   0.592593    0.363636   0.884615   0.818182  
'under' 0.500000   0.550000   0.333333   0.400000    0.444444   0.634146   0.588235
'over'  0.571429   0.428571   0.514286   0.666667    0.200      0.747475   0.684211

#shallow           # joint
'1pcs'  0.0        0.275862   0.093023   0.333333    0.133333   0.264706   0.335025
'5pcs'  0.060606   0.368421   0.354839   0.307692    0.266667   0.560847   0.382166
'10pcs' 0.500000   0.346667   0.333333   0.489362    0.466667   0.663317   0.484375
'full'  0.518519   0.574468   0.459016   0.606061    0.444444   0.941176   0.932331

                    #first
'full'  0.571429   0.448980   0.424242   0.500000    0.500      0.938776   0.916667
'under' 0.631579   0.342857   0.545455   0.578947    0.500000   0.786517   0.666667
'over'  0.461538   0.372093   0.526316   0.580645    0.400000   0.969072   0.927536

#alexnet
'1pcs'  0.0        0.038462   0.311475   0.0         0.160000   0.472527   0.301370  
'5pcs'  0.238806   0.338028   0.323529   0.195122    0.162162   0.380952   0.444444
'10pcs' 0.430769   0.296296   0.301887   0.507937    0.338983   0.589744   0.488889
'full'  0.645161   0.529412   0.327273   0.428571    0.514286   0.915423   0.878788
 
        #first'
'full'  0.500000   0.553191   0.388889   0.500000    0.428571   0.893617   0.788732
'under' 0.470588   0.476190   0.411765   0.482759    0.526316   0.704545   0.649351
'over'  0.714286   0.553191   0.4375     0.500000    0.714286   0.909091   0.777778

# Resnet           #first

'full'  0.400000   0.692308   0.500000   0.516129    0.571429   0.979592   0.941176
'under' 0.222222   0.564103   0.470588   0.461538    0.461538   0.758621   0.698413
'over'  0.166667   0.304348   0.375      0.486486    0.421053   0.698795   0.707692

In [ ]:
# overall 


'lay'      'pickup'   'sit'      'stand'     'standff'  'walk'    'waving'
0.666667   0.708333   0.6875     0.592593    0.363636   0.893204   0.835821  'simclr'  'first'  #vgg16
0.714286   0.415094   0.258065   0.592593    0.363636   0.884615   0.818182  'normal'    
0.421053   0.494382   0.327273   0.538462    0.648649   0.919431   0.876923            'joint'
0.500000   0.619048   0.600000   0.533333    0.400000   0.769231   0.724638            'l2reg'  


# comparison between models: larger model not always the best, clr 

'lay'      'pickup'   'sit'      'stand'     'standff'  'walk'    'waving'
0.571428   0.448979   0.424242   0.5         0.5        0.938775   0.916666  'shallow'  'normal'
0.500000   0.5        0.4375     0.580645    0.615385   0.923077   0.909091  'shallow'  'simclr' 

0.5        0.553191   0.388888   0.5         0.428571   0.893617   0.788732  'alexnet'  'normal'
0.875000   0.681818   0.611111   0.571429    0.75       0.907216   0.898551  'alexnet'  'simclr'

0.714286   0.415094   0.258065   0.592593    0.363636   0.884615   0.818182  'vgg16'    'normal'
0.666667   0.708333   0.6875     0.592593    0.363636   0.893204   0.835821  'vgg16'    'simclr'

0.400000   0.692308   0.500000   0.516129    0.571429   0.979592   0.941176  'resnet'   'normal'
0.400000   0.666667   0.611111   0.482759    0.428571   0.843137   0.823529  'resnet'   'simclr'


# effect on sample: simclr significantly improve sample efficiency 

'lay'      'pickup'   'sit'      'stand'     'standff'  'walk'    'waving'
0.210526   0.133333   0.2285714  0.266666    0.0        0.594059   0.148148  '1pcs'  'normal'   #vgg16
0.0,       0.638297   0.3783783  0.222222    0.142857   0.328767   0.479166  '1pcs'  'simclr'   

0.125      0.384615   0.3999999  0.32        0.230769   0.536082   0.369230  '5pcs'  'normal'
0.526315   0.627450   0.5        0.476190    0.133333   0.66       0.441176  '5pcs'  'simclr'

0.285714   0.333333   0.4705882  0.538461    0.434782   0.447368   0.430379  '10pcs' 'normal'
0.777777   0.651162   0.5714285  0.5         0.526315   0.64       0.578947  '10pcs' 'simclr'

0.714286   0.415094   0.258065   0.592593    0.363636   0.884615   0.818182  'full'  'normal'
0.666667   0.708333   0.6875     0.592593    0.363636   0.893204   0.835821  'full'  'simclr'

0.500000   0.550000   0.333333   0.400000    0.444444   0.634146   0.588235  'under' 'normal'
0.700000   0.714286   0.533333   0.482759    0.421053   0.755102   0.617647  'under' 'simclr'

0.571429   0.428571   0.514286   0.666667    0.200      0.747475   0.684211  'over'  'normal'
0.736842   0.681818   0.647059   0.642857    0.461538   0.88       0.823529  'over'  'simclr'


# effect on modality/view: seperate multiview is more beneficial to the training 
'lay'      'pickup'   'sit'      'stand'     'standff'  'walk'    'waving'
0.666667   0.708333   0.6875     0.592593    0.363636   0.893204   0.835821  'first'
0.437500   0.586957   0.426230   0.557377    0.400000   0.917874   0.883721  'joint'
0.0        0.775510   0.588235   0.666667    0.0        0.944000   0.833333  'pwr'


# effect on temperature: seperate multiview is more beneficial to the training 
'lay'      'pickup'   'sit'      'stand'     'standff'  'walk'    'waving'
0.666667   0.708333   0.6875     0.592593    0.363636   0.893204   0.835821   'L'
0.571429   0.653061   0.666667   0.538462    0.631579   0.921569   0.878788   'M'
0.428571   0.680851   0.647059   0.592593    0.588235   0.847826   0.826667   'H'


# activity 
'lay'      'pickup'   'sit'      'stand'     'standff'  'walk'    'waving'  

0.687500   0.666667   0.428571   0.622951               0.99                 #'vgg16' 'simclr'
0.727273   0.673267   0.407407   0.633333               0.942308   0.919355
0.514286   0.666667   0.448276   0.622951    0.370370   0.975369
0.437500   0.586957   0.426230   0.557377    0.400000   0.917874   0.883721

# Extra

In [ ]:
################################ Field-Finetuning-phase ################################
    
        for field_sampling in [1,5,10]:

            Xf,yf = import_data(fp2)
            field_finetune_loader, field_validatn_loader, lb = process_field_data(Xf,yf,num=field_sampling,lb=lb)
            
            
            model = load_model(network, freeze, lb, model_fp)
            
            # initialization
            phase = 'field-initial'
            if inital['field'] == True:
                cmtx,cls = evaluation(model,test_loader,label_encoder=lb)
                record_log(record_outpath,exp_name,phase,cmtx=cmtx,cls=cls)
                inital['field'] = False
                
            # finetuning 
            phase = 'field-finetune'
            criterion = nn.CrossEntropyLoss(weight=class_weight).to(device)
            optimizer = torch.optim.Adam(list(model.parameters()), lr=0.0005)
            model, record = train(model=model,
                                  train_loader= field_finetune_loader,
                                  criterion=criterion,
                                  optimizer=optimizer,
                                  end= field_finetune_epochs,
                                  test_loader = field_validatn_loader,
                                  device = device,
                                  regularize = regularize)
            # record and save
            evaluation(model,field_validatn_loader,label_encoder=lb)
            record_log(record_outpath,exp_name,phase,record=record,cmtx=cmtx,cls=cls,acc_rec=True)
            del encoder,model,record,cmtx,cls,criterion,optimizer
            torch.cuda.empty_cache()

        
        
        
    
    